In [1]:
import pandas as pd

from scipy.stats import ttest_ind

## Данные во время эксперимента из базы данных

```SQL
mysql -u Vladicheck -p -h Vladicheck.mysql.pythonanywhere-services.com -e 
"SELECT user_id, date, model_id, rating
FROM (
    SELECT request_id, user_id, date, model_id
    FROM requests
) t1 LEFT JOIN (
    SELECT request_id, rating
    FROM reviews
) t2 USING(request_id)"
'Vladicheck$ai_chef' > recsys_experiment.csv
```

In [2]:
df_exp = pd.read_csv('../data/recsys_experiment.csv')
df_exp.head()

,user_id,date,model_id,rating
0,1,2026-04-01 15:34:05,2,NaN
1,0,2026-04-01 15:44:14,1,3.0
2,1,2026-04-01 15:47:22,1,3.0
3,1,2026-04-01 16:11:52,1,NaN
4,0,2026-04-01 16:28:22,1,NaN


## Дизайн эксперимента

Гипотеза: Новая рекомендательная система с использование BERT повысит долю успешных рекомендаций на $0.1$%

- Уровень значимости: $\alpha = 0.05$
- Мощность: $1 - \beta = 0.9$
- Ожидаемый эффект: $0.1%$
- Ration метрика: Доля успешных рекомендаций (рейтинг $> 2$)
- Метод линеаризация

In [3]:
df_exp["rating"].fillna(0, inplace=True)

In [4]:
df_exp["success"] = df_exp["rating"] > 2
df_exp["success"] = df_exp["success"].astype(int)

In [5]:
ab_data = df_exp.groupby(["user_id", "model_id"]).agg(
        success_reviews=("success", "sum"),
        review_count=("success", "count")
    ).reset_index()

In [6]:
a_group, b_group = ab_data.query("model_id == 1"), ab_data.query("model_id == 2")

## Линеаризация метрики

In [7]:
a_group['success_reviews'].sum() / a_group['review_count'].sum(), \
    b_group['success_reviews'].sum() / b_group['review_count'].sum()

(0.39171974522292996, 0.5069984447900466)

In [8]:
k = a_group['success_reviews'].sum() / a_group['review_count'].sum()

In [9]:
a_linear = a_group['success_reviews'] - k * a_group['review_count']
b_linear = b_group['success_reviews'] - k * b_group['review_count']

In [10]:
test_result = ttest_ind(a_linear, b_linear)
test_result

TtestResult(statistic=-4.093915114997053, pvalue=5.858733993417231e-05, df=232.0)

In [11]:
test_result.pvalue < 0.05

True

## Результат эксперимента

Значение $pvalue < 0.05$.

Модель на основе BERT статистически значимо лучше, чем TF-IDF!